In [4]:
import tensorflow as tf
import numpy as np
import pickle

tf.compat.v1.disable_eager_execution()
# Create model
def create_multilayer_perceptron():
    # Network Parameters
    n_hidden_1 = 256  # 1st layer number of features
    n_hidden_2 = 256  # 2nd layer number of features
    n_input = 2376    # data input
    n_classes = 2     # number of classes

    # Store layers weight & bias
    weights = {
        'h1': tf.compat.v1.Variable(tf.random.normal([n_input, n_hidden_1])),
        'h2': tf.compat.v1.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
        'out': tf.compat.v1.Variable(tf.random.normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.compat.v1.Variable(tf.random.normal([n_hidden_1])),
        'b2': tf.compat.v1.Variable(tf.random.normal([n_hidden_2])),
        'out': tf.compat.v1.Variable(tf.random.normal([n_classes]))
    }
    
    # tf Graph input
    x = tf.compat.v1.placeholder("float", [None, n_input])
    y = tf.compat.v1.placeholder("float", [None, n_classes])

    # Hidden layer with ReLU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with ReLU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer, x, y


In [5]:
# Data preprocessing function
def preprocess():
    with open('basecode/face_all.pickle', 'rb') as file:
        pickle_obj = pickle.load(file)
    features = pickle_obj['Features']
    labels = pickle_obj['Labels']
    train_x = features[0:21100] / 255
    valid_x = features[21100:23765] / 255
    test_x = features[23765:] / 255

    labels = labels.T
    train_y = np.zeros(shape=(21100, 2))
    train_l = labels[0:21100]
    valid_y = np.zeros(shape=(2665, 2))
    valid_l = labels[21100:23765]
    test_y = np.zeros(shape=(2642, 2))
    test_l = labels[23765:]
    for i in range(train_y.shape[0]):
        train_y[i, train_l[i]] = 1
    for i in range(valid_y.shape[0]):
        valid_y[i, valid_l[i]] = 1
    for i in range(test_y.shape[0]):
        test_y[i, test_l[i]] = 1

    return train_x, train_y, valid_x, valid_y, test_x, test_y

In [6]:
# Parameters
learning_rate = 0.0001
training_epochs = 100
batch_size = 100

# Construct model
pred, x, y = create_multilayer_perceptron()

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.compat.v1.global_variables_initializer()

# Load data
train_features, train_labels, valid_features, valid_labels, test_features, test_labels = preprocess()

# Launch the graph in a session
with tf.compat.v1.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0] / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = train_features[i * batch_size: (i + 1) * batch_size]
            batch_y = train_labels[i * batch_size: (i + 1) * batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test Accuracy:", sess.run(accuracy, feed_dict={x: test_features, y: test_labels}))



I0000 00:00:1730071167.995763  365037 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


Epoch: 0001 cost=1419.765014648
Epoch: 0002 cost=264.745544434
Epoch: 0003 cost=210.054611206
Epoch: 0004 cost=177.743545532
Epoch: 0005 cost=155.156082153
Epoch: 0006 cost=137.998367310
Epoch: 0007 cost=124.820838928
Epoch: 0008 cost=114.100814819
Epoch: 0009 cost=105.536781311
Epoch: 0010 cost=98.392127991
Epoch: 0011 cost=91.974380493
Epoch: 0012 cost=86.371940613
Epoch: 0013 cost=80.825386047
Epoch: 0014 cost=76.470207214
Epoch: 0015 cost=72.471519470
Epoch: 0016 cost=69.011634827
Epoch: 0017 cost=65.767593384
Epoch: 0018 cost=62.760227203
Epoch: 0019 cost=60.108112335
Epoch: 0020 cost=57.695648193
Epoch: 0021 cost=55.427169800
Epoch: 0022 cost=53.419078827
Epoch: 0023 cost=51.416366577
Epoch: 0024 cost=49.710140228
Epoch: 0025 cost=47.917114258
Epoch: 0026 cost=46.390052795
Epoch: 0027 cost=44.884624481
Epoch: 0028 cost=43.640506744
Epoch: 0029 cost=42.231151581
Epoch: 0030 cost=40.997524261
Epoch: 0031 cost=39.641208649
Epoch: 0032 cost=38.558593750
Epoch: 0033 cost=37.414470673
